In [ ]:
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI, RateLimitError
import os

load_dotenv()

deployment_name='gpt4Test'
max_tokens=4096

client = AzureOpenAI(
    api_key=os.getenv("AZURE_API_KEY"),  
    api_version=os.getenv("AZURE_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT")
)

def generate_chat_gpt_query(prompt):
    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
        n=1,
        stop=None
    )

    if response.choices:
        return response.choices[0].message.content.strip()
    else:
        return "No suggestion generated."
    
df = pd.read_csv("dataset_with_sentiment.csv")

df_negative = df[df['label'] == 'negative']

specific_days = ['2023-01-02', '2022-12-16']

df_specific_days = df_negative[df_negative['day'].isin(specific_days)].drop(columns=["Unnamed: 0"])

context = df_specific_days.to_json(orient="records")

query = (
    "Given context contains negative feedbacks from my team received during sprint retrospectives from days: " + ", ".join(specific_days) + "\n"
    "1. What common themes or patterns do you detect in the negative feedback provided over the different days?\n"
    "2. Highlight feedbacks from the same topic which appear most frequently.\n"
)

prompt = "Context:\n" + context + "\n\n Prompt:\n" + query
print(prompt)
ai_response = generate_chat_gpt_query(prompt)
print(ai_response)
